<a href="https://colab.research.google.com/github/nathencinoza/pyspark-movie-dataset/blob/main/S26.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

S26). ¿Quién es el crew que trabajó en más tuplas (departamento, película) distintas?


In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
#!apt install default-jre
#!apt install default-jdk
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 47 kB/s 
     |████████████████████████████████| 199 kB 49.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=204211ebf26d332fa142930d42b4c9995e4f08f0e7514eac022ac329bda14918
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark
Ign:1 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://archive.ubuntu.com

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)
rdd_crew = sqlContext.read.parquet('/content/drive/MyDrive/Orga 2/Movie dataset/Parquets/crew.parquet').rdd
rdd_movies = sqlContext.read.parquet('/content/drive/MyDrive/Orga 2/Movie dataset/Parquets/movies.parquet').rdd

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:114: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning,


In [ ]:
rdd_movies = rdd_movies.map(lambda x: (x.id, x.title))

In [ ]:
rdd_crew = rdd_crew.map(lambda x: (str(x.id), (x.crew_id, x.department)))

In [ ]:
rdd = rdd_movies.join(rdd_crew).distinct()
rdd.take(5)

[('949', ('Heat', (638, 'Directing'))),
 ('949', ('Heat', (638, 'Writing'))),
 ('949', ('Heat', (11099, 'Camera'))),
 ('949', ('Heat', (15842, 'Editing'))),
 ('949', ('Heat', (897, 'Production')))]

Crew que trabajó en más tuplas (departamento, película) distintas:

In [ ]:
rdd.map(lambda x: (x[1][1][0],1)).reduceByKey(lambda x,y: x+y).max(key=lambda x: x[1])

(9062, 270)